In [1]:
#Import pandas
import pandas as pd

In [3]:
#import yelp csv to clean up
file_path='../Resources/yelpdata.csv'
yelp_df=pd.read_csv(file_path)
yelp_df.head()

,Unnamed: 0,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,0,0,38.046407,-84.497039,beaches,2,4.75,3.500
1,1,0,38.046407,-84.497039,horsebackriding,16,4.50,13.375
2,2,0,38.046407,-84.497039,hiking,8,4.50,8.500
3,3,0,38.046407,-84.497039,paddleboarding,2,4.75,16.000
4,4,0,38.046407,-84.497039,ziplining,20,5.00,8.450


In [4]:
#get rid of the unnamed column
yelp_df = yelp_df.drop(yelp_df.columns[0], axis=1)
yelp_df.head()

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,0,38.046407,-84.497039,beaches,2,4.75,3.500
1,0,38.046407,-84.497039,horsebackriding,16,4.50,13.375
2,0,38.046407,-84.497039,hiking,8,4.50,8.500
3,0,38.046407,-84.497039,paddleboarding,2,4.75,16.000
4,0,38.046407,-84.497039,ziplining,20,5.00,8.450


In [5]:
#brought over note from aggregate workbook, standardizing categories across datasets
# Clean final DF Activities to be one of the following list: 
#'beaches','hanggliding','horsebackriding', 'hiking','hot_air_balloons',
#'paddleboarding','parasailing','sailing','snorkeling','ziplining', 
#'Spas','hotsprings','massage','tours'

In [6]:
#fix spelling of balloons now, will match in other dataframe
yelp_df['category'].replace('hot_air_baloons','hot_air_balloons',inplace=True)

In [7]:
#Should effectively drop all rows where the category column does not contain one of the above values. 
yelp_df=yelp_df[yelp_df['category'].str.contains("beaches|hanggliding|horsebackriding|hiking|hot_air_balloons|paddleboarding|parasailing|sailing|snorkeling|ziplining|Spas|hotsprings|massage|tours")== True]

In [8]:
['Horsebackriding', 'Temples', 'Paddleboarding', 'nan', 'Whale Watching', 'Snorkeling', 'Parasailing', 'Tours', 
 'Wine Tours', 'Hanggliding', 'Beaches', 'Hot_Air_Balloons', 'Spas', 'Hot Springs', 'Sailing', 'Hiking', 'Massage', 'Ziplining', 'Hotsprings', 'Hot Air Balloons', 'Hang Gliding']



['Horsebackriding',
 'Temples',
 'Paddleboarding',
 'nan',
 'Whale Watching',
 'Snorkeling',
 'Parasailing',
 'Tours',
 'Wine Tours',
 'Hanggliding',
 'Beaches',
 'Hot_Air_Balloons',
 'Spas',
 'Hot Springs',
 'Sailing',
 'Hiking',
 'Massage',
 'Ziplining',
 'Hotsprings',
 'Hot Air Balloons',
 'Hang Gliding']

In [9]:
#create a 'top rated' column based on the average rating and number of reviews 
yelp_df['top_rated'] = yelp_df['avg_rating'] * yelp_df['avg_review_count'] 
yelp_df.head()

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count,top_rated
0,0,38.046407,-84.497039,beaches,2,4.75,3.500,16.6250
1,0,38.046407,-84.497039,horsebackriding,16,4.50,13.375,60.1875
2,0,38.046407,-84.497039,hiking,8,4.50,8.500,38.2500
3,0,38.046407,-84.497039,paddleboarding,2,4.75,16.000,76.0000
4,0,38.046407,-84.497039,ziplining,20,5.00,8.450,42.2500


In [10]:
#group by [lat,lon] pair and sort top rated value to descending
yelp_df = yelp_df.groupby(['latitude', 'longitude']).apply(lambda x: x.sort_values(by='top_rated', ascending=False))
yelp_df.head()

locationID   latitude  longitude     category  \
latitude   longitude                                                      
-45.032192 168.661   237          25 -45.032192    168.661   hotsprings   
                     235          25 -45.032192    168.661    ziplining   
                     243          25 -45.032192    168.661        tours   
                     238          25 -45.032192    168.661      massage   
                     232          25 -45.032192    168.661  hanggliding   

                          results_total  avg_rating  avg_review_count  \
latitude   longitude                                                    
-45.032192 168.661   237              1        4.50             20.00   
                     235             20        4.95             10.25   
                     243             20        4.80              4.45   
                     238              2        4.75              3.50   
                     232              1        4.50              3.00   

                          top_rated  
latitude   longitude                 
-45.032192 168.661   237    90.0000  
                     235    50.7375  
                     243    21.3600  
                     238    16.6250  
                     232    13.5000

In [11]:
#get rid of that index.
yelp_df=yelp_df.reset_index(drop=True)
yelp_df.head()

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count,top_rated
0,25,-45.032192,168.661,hotsprings,1,4.50,20.00,90.0000
1,25,-45.032192,168.661,ziplining,20,4.95,10.25,50.7375
2,25,-45.032192,168.661,tours,20,4.80,4.45,21.3600
3,25,-45.032192,168.661,massage,2,4.75,3.50,16.6250
4,25,-45.032192,168.661,hanggliding,1,4.50,3.00,13.5000


In [12]:
#group by again and drop all values below the top five for each [lat,lon] pair
yelp_df=yelp_df.groupby(['latitude','longitude']).head(5)
yelp_df.head(15)

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count,top_rated
0,25,-45.032192,168.661000,hotsprings,1,4.500000,20.000000,90.000000
1,25,-45.032192,168.661000,ziplining,20,4.950000,10.250000,50.737500
2,25,-45.032192,168.661000,tours,20,4.800000,4.450000,21.360000
3,25,-45.032192,168.661000,massage,2,4.750000,3.500000,16.625000
4,25,-45.032192,168.661000,hanggliding,1,4.500000,3.000000,13.500000
7,69,-37.814245,144.963173,ziplining,20,5.000000,6.950000,34.750000
8,69,-37.814245,144.963173,beaches,14,4.357143,4.357143,18.984694
9,69,-37.814245,144.963173,hiking,5,4.200000,4.200000,17.640000
10,69,-37.814245,144.963173,tours,20,4.850000,3.200000,15.520000
11,69,-37.814245,144.963173,hot_air_balloons,1,5.000000,3.000000,15.000000


In [13]:
# group again to convert the category column into a list and collapse the [lat,lon] pairs into a single row
grouped_df = yelp_df.groupby(['latitude', 'longitude']).agg({'category': lambda x: list(x)})
grouped_df.head()

,,category
latitude,longitude,
-45.032192,168.661000,"[hotsprings, ziplining, tours, massage, hanggl..."
-37.814245,144.963173,"[ziplining, beaches, hiking, tours, hot_air_ba..."
-37.754985,145.522537,"[hiking, ziplining, snorkeling, beaches, massage]"
-36.852095,174.763180,"[ziplining, beaches, tours, hiking, massage]"
-36.718805,142.220951,"[ziplining, tours]"


In [14]:
#loop through the locations and expand the category list into separate columns
for i in range(5):
    grouped_df[f'act{i+1}'] = grouped_df['category'].apply(lambda x: x[i] if len(x) > i else None)
grouped_df.head()

,,category,act1,act2,act3,act4,act5
latitude,longitude,,,,,,
-45.032192,168.661000,"[hotsprings, ziplining, tours, massage, hanggl...",hotsprings,ziplining,tours,massage,hanggliding
-37.814245,144.963173,"[ziplining, beaches, hiking, tours, hot_air_ba...",ziplining,beaches,hiking,tours,hot_air_balloons
-37.754985,145.522537,"[hiking, ziplining, snorkeling, beaches, massage]",hiking,ziplining,snorkeling,beaches,massage
-36.852095,174.763180,"[ziplining, beaches, tours, hiking, massage]",ziplining,beaches,tours,hiking,massage
-36.718805,142.220951,"[ziplining, tours]",ziplining,tours,None,None,None


In [15]:
#drop the original category column
grouped_df=grouped_df.drop(['category'],axis=1)
grouped_df.head()

,,act1,act2,act3,act4,act5
latitude,longitude,,,,,
-45.032192,168.661000,hotsprings,ziplining,tours,massage,hanggliding
-37.814245,144.963173,ziplining,beaches,hiking,tours,hot_air_balloons
-37.754985,145.522537,hiking,ziplining,snorkeling,beaches,massage
-36.852095,174.763180,ziplining,beaches,tours,hiking,massage
-36.718805,142.220951,ziplining,tours,None,None,None


In [16]:
yelp_df=yelp_df.reset_index(drop=True)
yelp_df.head()

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count,top_rated
0,25,-45.032192,168.661,hotsprings,1,4.50,20.00,90.0000
1,25,-45.032192,168.661,ziplining,20,4.95,10.25,50.7375
2,25,-45.032192,168.661,tours,20,4.80,4.45,21.3600
3,25,-45.032192,168.661,massage,2,4.75,3.50,16.6250
4,25,-45.032192,168.661,hanggliding,1,4.50,3.00,13.5000


In [17]:
### export it  as final_activities.csv
grouped_df.to_csv('.Resources/final_activities.csv',index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/final_activities.csv'